After running pip install pyngrok, go on terminal directory content and proceed to use these two commands to set up ngrok tunnelling


ngrok config add-authtoken 2ea0BVvbiDsWN7qKDcI1XD7UAPe_3nDgknKwJhpgK31pQxfBd


ngrok http --domain=humpback-smart-gently.ngrok-free.app 5000


In [1]:
!pip install pyngrok
!pip install flask-cors

In [2]:
!pip install -r /content/drive/MyDrive/requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.4/305.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 57.8 MB/s eta 0:00:00
  Created wheel for bson: filename=bson-0.5.10-py3-none-any.whl size=11976 sha256=cb7c563d2bb18403231d09d590c60bd08e51624418c62edfd467bb3ccb6e79ad
  Stored in directory: /root/.cache/pip/wheels/36/49/3b/8b33954dfae7a176009c4d721a45af56c8a9c1cdc3ee947945
Successfully built bson


In [ ]:
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2eeL3L1gyINdfSIp9Aa6I9iLpgh_7dkwxh9fmV1ASidjxvd6U"

ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok.kill()

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
public_url = ngrok.connect("5000")
print("Tracking URL:", public_url)

ModuleNotFoundError: No module named 'pyngrok'

In [4]:
from flask import render_template, request, redirect, flash, url_for, Response, jsonify, Flask
from bson import ObjectId
from flask_cors import CORS
from datetime import datetime
import tensorflow as tf
from tf2_yolov4.anchors import YOLOV4_ANCHORS
from tf2_yolov4.model import YOLOv4
import json
import cv2
import numpy as np
from io import BytesIO
from PIL import Image
import requests
import os
WIDTH, HEIGHT = (1024, 768)
app = Flask(__name__)
model = YOLOv4(
    input_shape=(HEIGHT, WIDTH, 3),
    anchors=YOLOV4_ANCHORS,
    num_classes=80,
    training=False,
    yolo_max_boxes=100,
    yolo_iou_threshold=0.5,
    yolo_score_threshold=0.30,
)
model.load_weights('/content/drive/MyDrive/Colab Notebooks/yolov4.h5')

highway401 = [
    'https://511on.ca/map/Cctv/loc21--2',
    'https://511on.ca/map/Cctv/loc51--2',
    'https://511on.ca/map/Cctv/loc52--2',
    'https://511on.ca/map/Cctv/loc21--2'
    ]

highway403 = [
    'https://511on.ca/map/Cctv/loc122--2',
    'https://511on.ca/map/Cctv/loc75--3',
    'https://511on.ca/map/Cctv/loc44--2',
    'https://511on.ca/map/Cctv/loc72--3'
]

highway404 = [
    'https://511on.ca/map/Cctv/loc138--2',
    'https://511on.ca/map/Cctv/loc55--2',
    'https://511on.ca/map/Cctv/loc141--2',
    'https://511on.ca/map/Cctv/loc136--2',
    'https://511on.ca/map/Cctv/loc140--2'
]

highway405 = [
    'https://511on.ca/map/Cctv/loc96--3'
]



highway410 = [
    'https://511on.ca/map/Cctv/loc75--2',
    'https://511on.ca/map/Cctv/loc827--2',
    'https://511on.ca/map/Cctv/loc825--2',
    'https://511on.ca/map/Cctv/loc826--2'
]


highway427=[
    'https://511on.ca/map/Cctv/loc701--2',
    'https://511on.ca/map/Cctv/loc702--2',
    'https://511on.ca/map/Cctv/loc703--2',
    'https://511on.ca/map/Cctv/loc718--2',
    'https://511on.ca/map/Cctv/loc725--2',
    'https://511on.ca/map/Cctv/loc46--3'
]

highwayQEW = [
    'https://511on.ca/map/Cctv/loc47--3',
    'https://511on.ca/map/Cctv/loc62--3',
    'https://511on.ca/map/Cctv/loc13--3',
    'https://511on.ca/map/Cctv/loc12--3',
    'https://511on.ca/map/Cctv/loc88--3'
]

highwayDVP = [
    'https://511on.ca/map/Cctv/mrr40fdbg1j--8',
    'https://511on.ca/map/Cctv/c3o4iglkxvw--8',
    'https://511on.ca/map/Cctv/loc57--2'
]

highwayGEW = [
    'https://511on.ca/map/Cctv/loc541--4',
    'https://511on.ca/map/Cctv/loc540--4',
    'https://511on.ca/map/Cctv/rf0j5dillxs--8',
    'https://511on.ca/map/Cctv/ac5ob4w4spc--8'
]

highway_data = {
    "401": highway401,
    "403": highway403,
    "404": highway404,
    "405": highway405,
    "410": highway410,
    "427": highway427,
    "QEW": highwayQEW,
    "DVP": highwayDVP,
    "GEW": highwayGEW
}

def load_image_to_tensor(file):
    # load image
    image = tf.io.read_file(file)
    # detect format (JPEG, PNG, BMP, or GIF) and converts to Tensor:
    image = tf.io.decode_image(image)
    return image

def resize_image(image):
    # Resize the output_image:
    image = tf.image.resize(image, (HEIGHT, WIDTH))
    # Add a batch dim:
    images = tf.expand_dims(image, axis=0)/255
    return images

def proccess_frame(photo, input_photo):
    images = resize_image(photo)
    boxes, scores, classes, detections = model.predict(images)
    result_img = detected_photo(boxes, scores, classes, detections,images[0],input_photo)
    return result_img


def detected_photo(boxes, scores, classes, detections,image,input_photo):
    boxes = (boxes[0] * [WIDTH, HEIGHT, WIDTH, HEIGHT]).astype(int)
    scores = scores[0]
    classes = classes[0].astype(int)
    detections = detections[0]

    CLASSES = [
        'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck',
        'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench',
        'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra',
        'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
        'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
        'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli',
        'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant',
        'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
        'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book',
        'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
    ]

    objects ={}
    obj_counter = 1
    image_cv = image.numpy()

    for (xmin, ymin, xmax, ymax), score, class_idx in zip(boxes, scores, classes):

        if score > 0:
            if class_idx == 2 or class_idx == 5 or class_idx == 7:         # show bounding box only to the "car,bus,truck" class

                #### Draw a rectangle ##################
                # convert from tf.Tensor to numpy
                cv2.rectangle(image_cv, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0,255,0), thickness= 2)
                # Add detection text to the prediction
                text = CLASSES[class_idx] + ': {0:.2f}'.format(score)
                cv2.putText(image_cv, text, (int(xmin), int(ymin) - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
                bounding_box = {
                    'xmin': int(xmin),
                    'ymin': int(ymin),
                    'xmax': int(xmax),
                    'ymax': int(ymax)
                }
                objects[f'Object{" "+str(obj_counter)}, bounding box'] = bounding_box
                obj_counter  += 1
    return objects

def calculate_roi_area(roi_coordinates):
    # Calculate area of the ROI defined by three points
    x1, y1 = roi_coordinates[0]
    x2, y2 = roi_coordinates[1]
    x3, y3 = roi_coordinates[2]

    area = abs((x1*(y2-y3) + x2*(y3-y1) + x3*(y1-y2)) / 2)
    return area

def calculate_roi_coordinates(bounding_boxes_data):

    if len(bounding_boxes_data)<5:
        roi_coordinates = [
            (0, 0),  # Bottom-left
            (0, 0),  # Bottom-right
            (0, 0)  # Top-middle
        ]
        return  roi_coordinates, len(bounding_boxes_data)

    all_x_coordinates = []
    all_y_coordinates = []

    # Parse bounding box data and collect all x and y coordinates
    for object_data in bounding_boxes_data.values():
        xmin = object_data["xmin"]
        ymin = object_data["ymin"]
        xmax = object_data["xmax"]
        ymax = object_data["ymax"]

        all_x_coordinates.extend([xmin, xmax])
        all_y_coordinates.extend([ymin, ymax])
    # Find extreme points
    min_x = min(all_x_coordinates)
    max_x = max(all_x_coordinates)
    min_y = min(all_y_coordinates)
    max_y = max(all_y_coordinates)


    if (len(bounding_boxes_data) <= 20):
        min_y = max(0,min_y-(0.1*min_y))
        max_y = min(768,max_y+(0.1*(max_y)))
    # Define ROI points: 2 at min y and 1 at max y
    roi_coordinates = [
        (min_x, min_y),  # Bottom-left
        (max_x, min_y),  # Bottom-right
        ((min_x + max_x) / 2, max_y)  # Top-middle
    ]

    return roi_coordinates, len(bounding_boxes_data)

def calculate_intersection_area(box, roi):
    # Calculate the intersection area between a bounding box and the ROI
    x_min = max(box[0], roi[0][0])
    y_min = max(box[1], roi[0][1])
    x_max = min(box[2], roi[1][0])
    y_max = min(box[3], roi[2][1])

    if x_max <= x_min or y_max <= y_min:
        return 0
    else:
        return (x_max - x_min) * (y_max - y_min)

def calculate_coverage_and_total_area(bounding_boxes_data, roi_coordinates):
    total_area_covered = 0
    for object_data in bounding_boxes_data.values():
        box = (object_data["xmin"], object_data["ymin"], object_data["xmax"], object_data["ymax"])
        intersection_area = calculate_intersection_area(box, roi_coordinates)
        total_area_covered += intersection_area

    roi_area = calculate_roi_area(roi_coordinates)
    coverage_percentage = (total_area_covered / roi_area) * 100 if roi_area > 0 else 0

    return roi_area, coverage_percentage

def read_bounding_boxes_from_txt(file_path):
    try:
        with open(file_path, 'r') as file:
            bounding_boxes_data = json.load(file)
        return bounding_boxes_data
    except FileNotFoundError:
        print("File not found.")
        return None
    except json.JSONDecodeError:
        print("Error decoding JSON.")
        return None

def get_coverage_data(url):
    response = requests.get(url)
    # Check if request was successful
    if response.status_code == 200:
        # Read the content of the response as bytes
        image_bytes = BytesIO(response.content)

        # Open the image using PIL
        image = Image.open(image_bytes)

        # Display the image
        image_path = "image.jpg"
        image.save(image_path)
    imagefile = 'image.jpg'

    my_image = load_image_to_tensor(imagefile)
    #Get trained yolov4 model
    image = proccess_frame(my_image,imagefile)
    roi, numberCars =calculate_roi_coordinates(image)
    os.remove(image_path)
    if all(coord == (0, 0) for coord in roi):
        return {
            'data':(0,0),
            'cars':numberCars
        }
    return {
        "data": calculate_coverage_and_total_area(image,roi),
        "cars": numberCars
    }

@app.route("/getHighwayCongestion",methods={'POST'})
def getHighwayCongestion():
    request_data = request.get_json()
    highway = request_data['highway']
    data = []
    if not highway in highway_data:
        return {
            'data': "Cameras not available",
            'message':'Invalid highway in database'
        },400
    try:
        if highway in highway_data:
            urls = highway_data[highway]
            for url in urls:
                coverage_percentage = get_coverage_data(url)
                data.append(coverage_percentage)
    except Exception as e:
        return {
                "message": "failed to run cv",
            }, 501
    sum_x = 0
    sum_y = 0
    count = 0
    minus_confidence=0
    # Calculate sum of x and y coordinates
    for item in data:
        x, y = item["data"]
        cars = item["cars"]
        if cars > 20:
            sum_x += x
            sum_y += y
            count +=1
        if cars > 9 and cars <= 20:
            sum_x += x
            sum_y += y*0.75
            count +=1
        if cars > 4 and cars <= 5:
            sum_x += x
            sum_y += y*0.5
            count +=1
        if cars == 0:
            minus_confidence+=1
            count+=1

    # Calculate average x and y coordinates
    if count-minus_confidence > 0:
        average_x = sum_x / (count-minus_confidence)
        average_y = sum_y / (count-minus_confidence)
    else:
        average_x = WIDTH*HEIGHT*0.25
        average_y = 0
    confidence = min(average_x/(WIDTH*HEIGHT*0.25),1)
    if count == 0:
        return {
            "data":(average_y,confidence),
            "message":"<30 indicate low, 30~45 indicate mild, 45%+ indicate high"
        },200

    if minus_confidence/count+minus_confidence > 0.4 and minus_confidence > 0:
        return {
            "data":(0,0),
            "message":"Road conditions too rough for CV"
        }, 200
    return {
        "data":(average_y,confidence),
        "message":"<30 indicate low, 30~45 indicate mild, 45%+ indicate high"
    },200
CORS(app)
app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


1/1 [==============================] - 0s 182ms/step


INFO:werkzeug:127.0.0.1 - - [05/Apr/2024 15:00:28] "POST /getHighwayCongestion HTTP/1.1" 200 -


1/1 [==============================] - 0s 148ms/step


...

INFO:werkzeug:127.0.0.1 - - [05/Apr/2024 15:03:55] "POST /getHighwayCongestion HTTP/1.1" 200 -


1/1 [==============================] - 0s 178ms/step


INFO:werkzeug:127.0.0.1 - - [05/Apr/2024 15:04:14] "POST /getHighwayCongestion HTTP/1.1" 200 -


1/1 [==============================] - 0s 174ms/step


INFO:werkzeug:127.0.0.1 - - [05/Apr/2024 15:04:39] "POST /getHighwayCongestion HTTP/1.1" 200 -
